In [37]:
# 타베로그 크롤링
import os.path
import time
import bs4
import csv
import datetime
import urllib
import json
import pymysql

In [51]:
page = 1
while page <=5:
    url = f'https://tabelog.com/tokyo/rstLst/{page}/?Srt=D&SrtT=rt&sort_mode=1&svd=20250220&svt=1900&svps=2'
    bs = bs4.BeautifulSoup(urllib.request.urlopen(url).read(), 'html.parser')
    shop_list_20 = bs.find_all('div', class_='list-rst__rst-data')
    shop_img_src_20 = bs.find_all('div', class_="list-rst__photo-item list-rst__photo-item--cover js-cassette-img js-switch-thumbnail-img")
    if page != 1 and os.path.exists(r"./tabelog_tokyo_rst.json"):
        with open(r"./tabelog_tokyo_rst.json","r",encoding="utf-8", newline="") as jf:
                try:
                    shop_list = json.load(jf)
                except json.decoder.JSONDecodeError:
                    print('error')
                    shop_list = []
    else:
        shop_list = []
    for shop,shop_img_src in zip(shop_list_20,shop_img_src_20):
        shop_title = shop.find('a', class_='list-rst__rst-name-target cpy-rst-name js-ranking-num')
        shop_describe = shop.find('p', class_='list-rst__pr-title cpy-pr-title')
        shop_rating = shop.find('span', class_='c-rating__val c-rating__val--strong list-rst__rating-val')
        shop_price = shop.find('span', class_='c-rating-v3__val')
        shop_data = {
        'title': shop_title.text,
        'describe': shop_describe.text.strip() if shop_describe else None,
        'rating': shop_rating.text if shop_rating else None,
        'price': shop_price.text if shop_price else None,
        'image_src': shop_img_src['data-original'] if shop_img_src else None,
        }
        shop_list.append(shop_data)
        with open(r"./tabelog_tokyo_rst.csv","w",encoding="utf-8", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=["title", "describe", "rating", "price", "image_src"])
            writer.writeheader()
            for one_shop in shop_list:
                writer.writerow(one_shop)
        with open(r"./tabelog_tokyo_rst.json","w",encoding="utf-8", newline="") as jf:
            json.dump(shop_list, jf, ensure_ascii=False, indent=4)
    print(f"{page}번째 저장 완료")
    time.sleep(5)
    page += 1
db = pymysql.connect(host='localhost', user='webuser', passwd='webuser', db='webdb')
cursor = db.cursor()
tuple_list = [(rtr['title'],rtr['describe'],rtr['rating'],rtr['price'],rtr['image_src']) for rtr in shop_list]
query = "INSERT INTO tabelogRestaurant (shop_name, shop_describe, rating, price,image_src) VALUES (%s, %s, %s, %s, %s)"
try:
    cursor.executemany(query, tuple_list)
    db.commit()
    print(f"db에 저장 완료")
except Exception as e:
    db.rollback()
    print(e)
cursor.close()
db.close()

1번째 저장 완료
2번째 저장 완료
3번째 저장 완료
4번째 저장 완료
5번째 저장 완료
db에 저장 완료
